In [ ]:
import import_ipynb
import Graph
import numpy as np
import pandas as pd
import time
from math import inf, sin, cos, asin, acos, pi, radians
import heapq as hq

In [ ]:
graph = Graph.make_graph('./roadnetwork_data/Ulsan_nodes.csv', './roadnetwork_data/Ulsan_edges.csv')

In [ ]:
# calculate heuristic
def heuristic(frm, to):
    R = 6400000 # m
    sigma = acos(sin(radians(frm.lat))*sin(radians(to.lat))
                 + cos(radians(frm.lat))*cos(radians(to.lat))*cos(radians(abs(frm.lon-to.lon))))
    heuristic = R*sigma
    return heuristic

In [ ]:
def bidirectional_astar(g, start, target):
    start_time = time.time()
    
    # Initialize for implementation
    open_lst = []
    open_lst_R = []
    closed_lst =[]
    hq.heapify(open_lst)
    hq.heapify(open_lst_R)
        
    for i in g.vert_dict.values():
        i.visited = False
        i.visited_R = False
        i.g_score = inf
        i.f_score = inf
        i.g_score_R = inf
        i.f_score_R = inf
        i.prev = None
        i.prev_R = None
        
    start = g.vert_dict[start]
    target = g.vert_dict[target]
    

    # Initialize start node
    start.g_score = 0
    start.f_score = heuristic(start, target)
    start.prev = -1
    start.visited = True
    closed_lst.append(start)
    
    target.g_score_R = 0
    target.f_score_R = heuristic(target, start)
    target.prev_R = -1
    target.visited_R = True
    closed_lst.append(target)
    
    current = start
    current_R = target
    
    state = 1
    
    while True:
        # forward search
        state += 1
        for near_vtx, near_vtx_cost in current.adjacent.items():
            if near_vtx.visited == True: continue
            # update forward score
            new_g_score = current.g_score + near_vtx_cost
            if near_vtx.g_score > new_g_score:
                near_vtx.g_score = new_g_score
                near_vtx.f_score = near_vtx.g_score + (heuristic(near_vtx, target) - heuristic(start, near_vtx))/2
                near_vtx.prev = current
                hq.heappush(open_lst, (near_vtx.f_score,near_vtx))
                
        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst)
        # Choose the next node
        current = nxt_node[1]
        current.visited = True
        closed_lst.append(current)
        if current.visited_R == True:
            node = current
            break

        # backward search
        state += 1
        for near_vtx, near_vtx_cost in current_R.adjacent_R.items():
            if near_vtx.visited_R == True: continue
            # update g_score
            new_g_score_R = current_R.g_score_R + near_vtx_cost
            if near_vtx.g_score_R > new_g_score_R:
                near_vtx.g_score_R = new_g_score_R
                near_vtx.f_score_R = near_vtx.g_score_R + (heuristic(start, near_vtx) - heuristic(near_vtx, target))/2
                near_vtx.prev_R = current_R
                hq.heappush(open_lst_R, (near_vtx.f_score_R,near_vtx))

        # By heappop current node is deleted from open list
        nxt_node = hq.heappop(open_lst_R)
        # Choose the next node
        current_R = nxt_node[1]
        current_R.visited_R = True
        closed_lst.append(current_R)
        if current_R.visited == True:
            node = current_R
            break

    # shortest forward path
    path_node = node
    path_f = [path_node.id]    
    while path_node != start:
        path_node = path_node.prev
        path_f.append(path_node.id)
    path_f.reverse()
    
    # shortest backward path
    path_node = node
    path_R = [path_node.id]
    while path_node != target:
        path_node = path_node.prev_R
        path_R.append(path_node.id)

    path = path_f+path_R[1:]

    execution_time = time.time() - start_time

    print("[A star with Consistent Approach]")
    print(">>> distance cost is:", node.g_score + node.g_score_R)
    print(">>> #iteration: ", state)
    print(">>> #expanded nodes:", len(closed_lst))
    print(">>> execution time: ", execution_time)
    
    return path, closed_lst

In [ ]:
path, closed_lst = bidirectional_astar(graph, 259986397, 9135501698)

In [ ]:
ans = np.array(path)
ans.size